
# Data Standardization and Flattening (JSON, XML, and XLSX)

This notebook is responsible for standardizing and flattening JSON, XML, and XLSX files. It reads the raw data from the landing zone, applies schemas for validation where applicable, flattens nested structures using `depth_level`, and then writes the transformed data as Delta Parquet files.

## Key Features:
- **Multi-format Support:**
  - Reads JSON, XML, and XLSX files from the landing zone.
  - Handles nested structures and various data types and formats.

- **Schema Validation:**
  - Applies predefined schemas for data validation:
    - JSON: Schema files must be available at `landing/schemachecks/[datasetidentifier]/[datasetidentifier]_schema.json`.
    - XML: Validates against XSD (XML Schema Definition) files.
    - XLSX: Schema validation is not applicable.

- **Data Flattening:**
  - Supports flattening of nested structures in JSON and XML using `depth_level` for controlling the hierarchy level to flatten.
  - Processes XLSX files into a structured, normalized format.

- **Efficient Data Storage:**
  - Saves the processed data as Delta Parquet files for efficient storage and querying.

This notebook provides a flexible and robust framework for standardizing and preparing data for downstream analytics across multiple file formats.

---

## ADF Pipeline Integration
- The notebook is designed to work seamlessly with Azure Data Factory (ADF) pipelines.
- Parameters such as `SourceDatasetidentifier`, `SourceStorageAccount`, and other configurations are dynamically passed from the ADF pipeline at runtime.
- External parameters passed by the pipeline automatically override the default or test configurations defined in the notebook.

---

## Function Definitions
- Most of the functions used in this notebook are part of a reusable library hosted on GitHub: [Open-Dataplatform/utils-databricks](https://github.com/Open-Dataplatform/utils-databricks).
- Detailed descriptions of these functions, including their purpose and usage, can be found in the GitHub repository.
- This allows you to reuse and extend the existing functionalities in your own workflows efficiently.

---

## Use Guide
1. **Clone the Notebook:**
   - Clone this notebook and use it as a template for your data processing workflows.

2. **Customize the Code:**
   - Remove unnecessary code or datasets that are not relevant to your use case.
   - Add custom logic and transformations based on your specific requirements.

3. **Test and Validate:**
   - Use the predefined dataset configurations as an inspiration (`triton__flow_plans`, `cpx_so__nomination`, etc.) to test and validate your notebook logic.

4. **Run in Production:**
   - Integrate this notebook into an ADF pipeline and pass the required parameters dynamically for production workflows.

This notebook serves as a reusable and customizable framework for handling multi-format data standardization and flattening tasks.

## Setup

### Package Installation and Management

In [0]:
# ==============================================================
# Setup: Package Installation and Management
# ==============================================================

# Purpose:
# Manage and install essential Python packages for the Databricks project.
# Ensures compatibility by specifying exact package versions where necessary.
# Includes support for utilities, data processing, and XML/XLSX handling.

# Step 1: Optional - Remove an existing version of the custom utility package
# Uncomment the line below if a previous version of the utility needs to be removed.
# %pip uninstall databricks-custom-utils -y

# Step 2: Install required packages
# The command below installs:
# - Custom Databricks utilities (specific version from GitHub repository).
# - Libraries for SQL parsing, Excel file handling, XML processing, and syntax highlighting.
%pip install \
    git+https://github.com/Open-Dataplatform/utils-databricks.git@v1.1.3 \
    sqlparse \
    openpyxl \
    lxml \
    xmlschema \
    pygments

"""
Package Details:
- `utils-databricks`: Custom utilities for extended functionality in Databricks.
- `sqlparse`: SQL query parsing and formatting library.
- `openpyxl`: Library for handling Excel (XLSX) files.
- `lxml`: Robust library for processing XML and HTML files.
- `xmlschema`: Tools for XML schema validation and conversion.
- `pygments`: Syntax highlighting for code snippets in logs or reports.
"""

## Initialization and Configuration

### Initialize Logger

In [0]:
# ==============================================================
# Initialize Logger
# ==============================================================

# Purpose:
# Set up a custom logger for detailed logging and debugging throughout the notebook.
# The logger offers advanced features, including:
# - Debug-level logging for in-depth insights during execution.
# - Block-style logging for structured, readable logs.
# - Syntax highlighting for SQL queries and Python code in logs.

# Step 1: Import the Logger class from the custom utilities package
from custom_utils.logging.logger import Logger

# Step 2: Initialize the Logger instance
# - `debug=True` enables detailed logs, useful for troubleshooting and analysis.
logger = Logger(debug=True)

# Log the initialization success
logger.log_message("Logger initialized successfully.")

### Initializes widgets

In [0]:
# ==============================================================
# Widget Initialization and Test Configuration
# ==============================================================

# Purpose:
# Set up widgets for testing the "xxx" dataset using XML files.
# The widgets enable dynamic configuration of critical parameters, including file type,
# storage accounts, containers, dataset identifiers, and XML-specific options.

# Step 1: Initialize File Type dropdown with supported options
dbutils.widgets.text("FileType", "json", "File Type")

# Step 2: Define Source and Destination Storage Accounts
dbutils.widgets.text("SourceStorageAccount", "xxx", "Source Storage Account")
dbutils.widgets.text("DestinationStorageAccount", "xxx", "Destination Storage Account")

# Step 3: Configure Source Container and Dataset Identifier
dbutils.widgets.text("SourceContainer", "xxx", "Source Container")
dbutils.widgets.text("SourceDatasetidentifier", "xxx", "Source Datasetidentifier")

# Step 4: Specify Source File Name and Key Columns
dbutils.widgets.text("SourceFileName", "xxx*", "Source File Name")
dbutils.widgets.text("KeyColumns", "xxx, yyy", "Key Columns")

# Step 5: Set Additional Parameters for Feedback Column, Depth Level, Schema Folder, and XML Root Name
dbutils.widgets.text("FeedbackColumn", "xxx", "Feedback Column")
dbutils.widgets.text("DepthLevel", "x", "Depth Level")
dbutils.widgets.text("SchemaFolderName", "xxx", "Schema Folder Name")

# Log the initialization success
logger.log_message("Widget initialized successfully.")

### Initialize notebook and retrieve parameters

In [0]:
# ==============================================================
# Initialize Notebook and Retrieve Parameters
# ==============================================================

# Purpose:
# Set up the notebook by initializing its configuration and retrieving essential parameters.
# This ensures centralized management of settings and enables efficient debugging
# through a consistent configuration framework.

# Step 1: Import the Config class from the custom utilities package
from custom_utils.config.config import Config

# Step 2: Initialize the Config object
# - Pass `dbutils` for accessing Databricks workspace resources.
# - Set `debug=False` to disable verbose debug logs for cleaner execution.
config = Config.initialize(dbutils=dbutils, debug=False)

# Step 3: Unpack configuration parameters
# - Extracts configuration values into the notebook's global scope.
# - This simplifies access to parameters by making them available as standard variables.
config.unpack(globals())

### Verify paths and files

In [0]:
# ==============================================================
# Verify Paths and Files
# ==============================================================

# Purpose:
# Validate the required paths and files to ensure all necessary resources 
# are available for processing. This pre-check prevents runtime errors 
# by identifying and addressing issues early in the notebook execution.

# Step 1: Import the Validator class from the custom utilities package
from custom_utils.validation.validation import Validator

# Step 2: Initialize the Validator
# - Pass `config` to access path and file parameters from the configuration.
# - Set `debug=False` for standard validation logging without verbose output.
validator = Validator(config=config, debug=False)

# Step 3: Unpack validation parameters
# - Extracts validation-related parameters into the notebook's global scope.
validator.unpack(globals())

# Step 4: Perform validation and check for an exit flag
# - If critical validation fails, the notebook execution is terminated.
validator.check_and_exit()


### Exit the Notebook if Validation Fails

In [0]:
# ==============================================================
# Exit the Notebook if Validation Fails
# ==============================================================

# Purpose:
# Stop notebook execution gracefully if critical validation checks fail.
# If validation passes, continue processing with a confirmation message.

# Step 1: Check for an exit condition flagged by the Validator
if Validator.exit_notebook:
    # Step 2: Log the exit message using the logger
    # - Provides context on why the notebook execution is being terminated.
    logger.log_error(Validator.exit_notebook_message)
    
    # Step 3: Exit the notebook with a descriptive message
    # - Uses Databricks utilities to terminate execution cleanly.
    dbutils.notebook.exit(f"Notebook exited: {Validator.exit_notebook_message}")
else:
    # Step 4: Log a success message if validation passed
    # - Confirms the notebook will continue execution.
    logger.log_message("Validation passed. The notebook is proceeding without exiting.")

## Processing Workflow

### Flattening and Processing

In [0]:
# ==============================================================
# Processing Workflow - Flattening and Processing
# ==============================================================

# Purpose:
# This workflow simplifies hierarchical data by flattening it, renaming columns, 
# casting data types, and applying dataset-specific transformations.

import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr, unix_timestamp, from_unixtime, to_utc_timestamp
from custom_utils.transformations.dataframe import DataFrameTransformer

# Initialize the DataFrameTransformer
transformer = DataFrameTransformer(config=config, debug=False)

try:
    # Step 1: Process and flatten the data
    df_initial, df_flattened = transformer.process_and_flatten_data(depth_level=depth_level)

    # Step 3: Display the flattened DataFrame
    logger.log_block("Displaying the flattened DataFrame.")
    display(df_flattened)

except Exception as e:
    # Handle errors gracefully
    logger.log_error(f"Error during processing: {str(e)}")
    dbutils.notebook.exit(f"Processing failed: {str(e)}")

## Quality check 

### Perform Quality Check and Remove Duplicates

In [0]:
# ==============================================================
# Quality Check - Perform Quality Check and Remove Duplicates
# ==============================================================

# Purpose:
# This section performs data quality checks to ensure:
# - The integrity, accuracy, and consistency of the processed data.
# - Duplicate records are identified and optionally removed.
# - Additional quality checks (e.g., null value checks, value range checks) are executed.

from custom_utils.quality.quality import DataQualityManager

# Step 1: Initialize the DataQualityManager
# - This class manages all quality check operations and logs relevant information.
quality_manager = DataQualityManager(logger=logger, debug=True)

# Step 2: Log available quality checks
# - Provides an overview of checks supported by the quality manager for user reference.
quality_manager.describe_available_checks()

# Step 3: Execute data quality checks on the flattened DataFrame
try:
    # Perform quality checks with the following configurations:
    cleaned_data_view = quality_manager.perform_data_quality_checks(
        spark=spark,  # Required: Spark session.
        df=df_flattened,  # Required: DataFrame to perform quality checks on.
        
        # Key columns for partitioning and duplicate checking.
        # - Required parameter to identify unique records in the dataset.
        key_columns=key_columns,
        
        # Optional: Columns for ordering within partitions (e.g., to select the latest record).
        # - Defaults to `key_columns` if not provided.
        order_by="input_file_name",
        
        # Optional: Column to use for duplicate removal ordering.
        # - If not provided, falls back to `key_columns`.
        feedback_column="input_file_name",
        
        # Optional: Column for referential integrity check against a reference DataFrame.
        # - Ensures that foreign key relationships are maintained.
        join_column=key_columns,
        
        # Optional: Exclude specified columns from the final DataFrame.
        # - For example, `input_file_name` is excluded to avoid irrelevant data in output.
        columns_to_exclude=["input_file_name"],
        
        # Optional: Specify whether to use Python or SQL syntax for quality checks.
        # - Default is SQL-based for optimized performance.
        use_python=False
    )

    # Description of Arguments:
    # - `spark`: Spark session used for distributed processing (required).
    # - `df`: The DataFrame on which quality checks are performed (required).
    # - `key_columns`: Columns used for identifying unique records (required).
    # - `order_by`: Columns for ordering within partitions (optional; defaults to `key_columns`).
    # - `feedback_column`: Column used for ordering duplicates (optional; falls back to `key_columns`).
    # - `join_column`: Column for referential integrity validation (optional).
    # - `columns_to_exclude`: List of columns to exclude from the final DataFrame (optional).
    # - `use_python`: Boolean flag to select Python-based or SQL-based operations (optional).

except Exception as e:
    # Handle any errors during the quality check process
    logger.log_error(f"Error during quality check: {str(e)}")
    raise RuntimeError(f"Quality check failed: {str(e)}")

## Unified Data Management

### Table Creation and Data Merging

In [0]:
# ==============================================================
# Unified Data Management: Table Creation and Data Merging
# ==============================================================

# Purpose:
# This section handles the creation of destination tables and merges
# processed data into the respective storage location. It ensures:
# - Data is written to a unified storage with consistent formatting.
# - Merging supports updates, inserts, and deletions seamlessly.
# - Storage operations are managed efficiently with robust logging.

from custom_utils.catalog.catalog_utils import DataStorageManager

# Step 1: Initialize the DataStorageManager
# - Manages operations related to data storage and merging.
# - Includes detailed logging and debugging capabilities.
storage_manager = DataStorageManager(logger=logger, debug=True)

# Step 2: Perform the data storage operation
try:
    # Manage data operation with the following configurations (these parameters are defined in the configuration and passed as parameters to the function, and can be overridden if necessary):
    # The used variables (cleaned_data_view, key_columns, etc.) are created and assigned by call to config.unpack(globals()) above, under section "Initialize Notebook and Retrieve Parameters"
    storage_manager.manage_data_operation(
        spark=spark,  # Required: Spark session for executing SQL or DataFrame operations.
        dbutils=dbutils,  # Required: Databricks utilities for interacting with storage.

        # Name of the cleaned data view containing the processed DataFrame.
        # - Required parameter that holds the cleaned and transformed data.
        cleaned_data_view=cleaned_data_view,

        # Key columns used for matching records during the merge operation.
        # - Required parameter to ensure data consistency during updates.
        key_columns=key_columns,

        # Destination folder path for storing data as Delta files.
        # - Optional: If not provided, a default path defined in the configuration is used.
        destination_folder_path=destination_data_folder_path,

        # Target database or environment for storing the data.
        # - Optional: Overrides the default destination environment if provided.
        destination_environment=destination_environment,

        # Target table name or identifier for the source dataset.
        # - Optional: Allows dynamic specification of the target table for merging.
        source_datasetidentifier=source_datasetidentifier,

        # Boolean flag to select SQL-based (default) or Python DataFrame-based operations.
        # - Optional: Default is `False` to prioritize SQL for performance.
        use_python=False
    )

    # Description of Arguments:
    # - `spark`: Active Spark session (required).
    # - `dbutils`: Databricks utilities object for workspace interaction (required).
    # - `cleaned_data_view`: Name of the view containing cleaned data (required).
    # - `key_columns`: Columns used for identifying unique records during merge (required).
    # - `destination_folder_path`: Override for the destination folder (optional).
    # - `destination_environment`: Override for the target database/environment (optional).
    # - `source_datasetidentifier`: Override for the source dataset/table identifier (optional).
    # - `use_python`: Boolean flag for using Python or SQL operations (optional).

    # Log success
    logger.log_message("Data successfully written and merged into the destination table.")

except Exception as e:
    # Step 3: Handle errors during the data storage process
    # - Logs the error details and raises a RuntimeError to terminate execution.
    logger.log_error(f"Error during data storage operation: {str(e)}")
    raise RuntimeError(f"Data storage operation failed: {str(e)}")

## Finishing

### Return period (from_datetime, to_datetime) covered by data read

In [0]:
# ==============================================================
# Finishing - Return Period Covered by Data Read
# ==============================================================

# Purpose:
# This section generates the feedback timestamps, providing the time 
# period covered by the processed and stored data. It calculates 
# `from_datetime` and `to_datetime` based on the data in the cleaned 
# data view.

# Step 1: Generate feedback timestamps
try:
    # Call the `generate_feedback_timestamps` function with the following:
    # - The active Spark session to access the view.
    # - The name of the cleaned data view containing processed data.
    # - The `feedback_column` for timestamp calculations (optional).
    # - The `key_columns` for grouping (optional).
    notebook_output = storage_manager.generate_feedback_timestamps(
        spark=spark,  # Active Spark session
        view_name=cleaned_data_view,  # The view containing cleaned and processed data
        feedback_column=feedback_column,  # Column used for identifying feedback periods
        key_columns=key_columns  # Key columns for grouping and extracting timestamp bounds
    )

    # Log the successful generation of feedback timestamps
    logger.log_message("Feedback timestamps successfully generated.", level="info")

except Exception as e:
    # Handle errors during feedback timestamp generation
    logger.log_error(f"Error generating feedback timestamps: {str(e)}")
    raise RuntimeError(f"Failed to generate feedback timestamps: {str(e)}")

## Exit the notebook

In [0]:
# ==============================================================
# Exit the Notebook
# ==============================================================

# Purpose:
# Finalize the notebook execution by exiting and returning the output.
# The output provides a summary of the period covered by the processed data,
# ensuring a clear handoff to any downstream workflows.

# Step 1: Exit the notebook with the generated output
try:
    # Use dbutils to exit the notebook gracefully
    # - The `notebook_output` contains feedback timestamps or relevant results.
    dbutils.notebook.exit(notebook_output)

    # Log the successful exit for tracking and debugging purposes
    logger.log_message(f"Notebook exited successfully with output: {notebook_output}", level="info")

except Exception as e:
    # Handle errors during the exit process
    # - Logs the error and raises a RuntimeError to signal failure.
    logger.log_error(f"Error during notebook exit: {str(e)}")
    raise RuntimeError(f"Failed to exit the notebook: {str(e)}")